In [1]:
%pwd

'c:\\Users\\peera\\Desktop\\DroughtLSTM_oneday\\visualize'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
spei_csv = pd.read_csv("./results/SPEI_ConvLSTM_Direct/global_convlstm_improved_full_predictions.csv")

In [3]:
spei_csv.head()

,time,lat,lon,spei,spei_predicted,prediction_error,absolute_error
0,1902-01-16,6.25,101.25,-0.322998,-0.075869,0.247129,0.247129
1,1902-01-16,6.25,101.75,-0.347505,-0.061522,0.285983,0.285983
2,1902-01-16,6.75,99.75,-0.164277,-0.184659,-0.020382,0.020382
3,1902-01-16,6.75,100.25,-0.242470,-0.174811,0.067659,0.067659
4,1902-01-16,6.75,100.75,-0.519982,-0.182955,0.337027,0.337027


In [4]:
spei_csv.shape

(262056, 7)

In [57]:
df = pd.read_csv("full.csv")
#create robust scaler
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [58]:
df.isnull().sum()

lon       0
lat       0
time      0
tmp       0
dtr       0
cld       0
tmx       0
tmn       0
pre       0
wet       0
vap       0
spei      0
soi       0
dmi       0
pdo       0
nino4     0
nino34    0
nino3     0
pet       0
dtype: int64

In [6]:
#robust scaler fit on time <=2017-12-31 df
df_train = df[["time","spei"]]


In [7]:
df_train.head()

,time,spei
0,1901-01-16,-0.384595
1,1901-01-16,-0.324920
2,1901-01-16,-0.612856
3,1901-01-16,-0.581451
4,1901-01-16,-0.330006


In [8]:
scaler.fit(df_train["spei"].values.reshape(-1, 1))

RobustScaler()

In [9]:
# create a new column in spei_csv with the inverse scaled values
spei_df = spei_csv.copy()
spei_df.drop(columns=['spei',"spei_predicted"], inplace=True, errors='ignore')
spei_csv['spei_inverse_scaled'] = scaler.inverse_transform(spei_csv['spei'].values.reshape(-1, 1))
spei_csv['spei_predicted_inverse_scaled'] = scaler.inverse_transform(spei_csv['spei_predicted'].values.reshape(-1, 1))
# save the updated dataframe to a new csv file



In [10]:
spei_csv

,time,lat,lon,spei,spei_predicted,prediction_error,absolute_error,spei_inverse_scaled,spei_predicted_inverse_scaled
0,1902-01-16,6.25,101.25,-0.322998,-0.075869,0.247129,0.247129,-0.457194,-0.106182
1,1902-01-16,6.25,101.75,-0.347505,-0.061522,0.285983,0.285983,-0.492003,-0.085805
2,1902-01-16,6.75,99.75,-0.164277,-0.184659,-0.020382,0.020382,-0.231754,-0.260704
3,1902-01-16,6.75,100.25,-0.242470,-0.174811,0.067659,0.067659,-0.342816,-0.246716
4,1902-01-16,6.75,100.75,-0.519982,-0.182955,0.337027,0.337027,-0.736983,-0.258284
...,...,...,...,...,...,...,...,...,...
262051,2023-12-16,19.75,99.25,-0.796599,-0.315188,0.481411,0.481411,-1.129878,-0.446102
262052,2023-12-16,19.75,99.75,-0.935252,-0.313575,0.621677,0.621677,-1.326815,-0.443810
262053,2023-12-16,19.75,100.25,-0.956201,-0.297494,0.658706,0.658706,-1.356570,-0.420970
262054,2023-12-16,20.25,99.75,-0.747644,-0.198313,0.549331,0.549331,-1.060345,-0.280098


In [14]:
def calculate_metrics(df):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    y_true = df['spei'].values
    y_pred = df['spei_predicted'].values

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2
    }
calculate_metrics(spei_csv)

{'rmse': np.float64(0.9802567136730649),
 'mae': np.float64(0.7928138443652417),
 'r2': 0.03510311541385103}

In [11]:
spei_df

,time,lat,lon,prediction_error,absolute_error
0,1902-01-16,6.25,101.25,0.247129,0.247129
1,1902-01-16,6.25,101.75,0.285983,0.285983
2,1902-01-16,6.75,99.75,-0.020382,0.020382
3,1902-01-16,6.75,100.25,0.067659,0.067659
4,1902-01-16,6.75,100.75,0.337027,0.337027
...,...,...,...,...,...
262051,2023-12-16,19.75,99.25,0.481411,0.481411
262052,2023-12-16,19.75,99.75,0.621677,0.621677
262053,2023-12-16,19.75,100.25,0.658706,0.658706
262054,2023-12-16,20.25,99.75,0.549331,0.549331


In [12]:
spei_df["spei"] = spei_csv['spei_inverse_scaled']
spei_df["spei_predicted"] = spei_csv['spei_predicted_inverse_scaled']

In [13]:
spei_df

,time,lat,lon,prediction_error,absolute_error,spei,spei_predicted
0,1902-01-16,6.25,101.25,0.247129,0.247129,-0.457194,-0.106182
1,1902-01-16,6.25,101.75,0.285983,0.285983,-0.492003,-0.085805
2,1902-01-16,6.75,99.75,-0.020382,0.020382,-0.231754,-0.260704
3,1902-01-16,6.75,100.25,0.067659,0.067659,-0.342816,-0.246716
4,1902-01-16,6.75,100.75,0.337027,0.337027,-0.736983,-0.258284
...,...,...,...,...,...,...,...
262051,2023-12-16,19.75,99.25,0.481411,0.481411,-1.129878,-0.446102
262052,2023-12-16,19.75,99.75,0.621677,0.621677,-1.326815,-0.443810
262053,2023-12-16,19.75,100.25,0.658706,0.658706,-1.356570,-0.420970
262054,2023-12-16,20.25,99.75,0.549331,0.549331,-1.060345,-0.280098


In [78]:
def calculate_metrics(df):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    # Remove rows with NaN or inf in either column
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['spei', 'spei_predicted'])
    y_true = df['spei'].values
    y_pred = df['spei_predicted'].values

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2
    }

In [83]:
spei_multi = pd.read_csv('results/spei_multiconv.csv')
spei_single = pd.read_csv('results/spei_singleconv.csv')
spei_raw = pd.read_csv('full.csv')
spei_raw.head()
# Ensure 'time' columns are in datetime format for accurate merging
spei_multi['time'] = pd.to_datetime(spei_multi['time'])
#rename the 'spei' column to 'spei_predicted'
spei_multi.rename(columns={'spei': 'spei_predicted'}, inplace=True)
spei_single.rename(columns={'spei': 'spei_predicted'}, inplace=True)
spei_single.dropna(inplace= True)
spei_multi.dropna(inplace= True)


In [84]:
# Ensure 'time' columns are in datetime format for accurate merging
spei_multi['time'] = pd.to_datetime(spei_multi['time'])
spei_single['time'] = pd.to_datetime(spei_single['time'])
# Rename the 'spei' column to 'spei_predicted'
spei_raw['time'] = pd.to_datetime(spei_raw['time'])

# Only keep necessary columns for merging
spei_raw_subset = spei_raw[['lat', 'lon', 'time', 'spei']]

# Merge on ['lat', 'lon', 'time']
spei_multi = spei_multi.merge(spei_raw_subset, on=['lat', 'lon', 'time'], how='left')
spei_single = spei_single.merge(spei_raw_subset, on=['lat', 'lon', 'time'], how='left')

# Preview result
print(spei_multi.head())

        time   lat     lon         pre  pre_predicted  pre_prediction_error  \
0 1902-01-16  6.25  101.25   81.000000       81.52027              0.520271   
1 1902-01-16  6.25  101.75  127.000000      141.74022             14.740219   
2 1902-01-16  6.75   99.75   43.600002       49.50343              5.903427   
3 1902-01-16  6.75  100.25   40.500000       46.68722              6.187221   
4 1902-01-16  6.75  100.75   43.100002       50.81467              7.714668   

   pre_absolute_error    pet  pet_predicted  pet_prediction_error  \
0            0.520271  102.3     106.799460              4.499458   
1           14.740219   99.2     104.200905              5.000908   
2            5.903427  127.1     129.733500              2.633507   
3            6.187221  117.8     119.497795              1.697792   
4            7.714668  108.5     113.688970              5.188973   

   pet_absolute_error  spei_predicted      spei  
0            4.499458       -0.462184 -0.322998  
1         

In [85]:
spei_multi.dropna(inplace=True)
spei_single.dropna(inplace=True)

In [73]:
spei_multi.isnull().sum()

time                    0
lat                     0
lon                     0
pre                     0
pre_predicted           0
pre_prediction_error    0
pre_absolute_error      0
pet                     0
pet_predicted           0
pet_prediction_error    0
pet_absolute_error      0
spei_predicted          0
spei                    0
dtype: int64

In [75]:
train.isnull().sum()

time                    0
lat                     0
lon                     0
pre                     0
pre_predicted           0
pre_prediction_error    0
pre_absolute_error      0
pet                     0
pet_predicted           0
pet_prediction_error    0
pet_absolute_error      0
spei_predicted          0
spei                    0
dtype: int64

In [82]:
train

,time,lat,lon,pre,pre_predicted,pre_prediction_error,pre_absolute_error,pet,pet_predicted,pet_prediction_error,pet_absolute_error,spei_predicted,spei_x,spei_y
0,1902-01-16,6.25,101.25,81.000000,81.520270,0.520271,0.520271,102.3,106.799460,4.499458,4.499458,-0.462184,-0.322998,-0.322998
1,1902-01-16,6.25,101.75,127.000000,141.740220,14.740219,14.740219,99.2,104.200905,5.000908,5.000908,-0.103014,-0.347505,-0.347505
2,1902-01-16,6.75,99.75,43.600002,49.503430,5.903427,5.903427,127.1,129.733500,2.633507,2.633507,0.112876,-0.164277,-0.164277
3,1902-01-16,6.75,100.25,40.500000,46.687220,6.187221,6.187221,117.8,119.497795,1.697792,1.697792,0.113577,-0.242470,-0.242470
4,1902-01-16,6.75,100.75,43.100002,50.814670,7.714668,7.714668,108.5,113.688970,5.188973,5.188973,-0.340741,-0.519982,-0.519982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247012,2016-12-16,19.75,99.25,1.800000,8.603022,6.803021,6.803021,71.3,73.461494,2.161491,2.161491,-1.044592,-0.633215,-0.633215
247013,2016-12-16,19.75,99.75,4.200000,8.940071,4.740071,4.740071,71.3,80.870970,9.570969,9.570969,-2.185282,-0.471951,-0.471951
247014,2016-12-16,19.75,100.25,5.600000,12.556427,6.956427,6.956427,71.3,71.781456,0.481453,0.481453,-0.319454,-0.360703,-0.360703
247015,2016-12-16,20.25,99.75,3.000000,8.990468,5.990468,5.990468,71.3,73.035890,1.735886,1.735886,-1.598241,-0.772386,-0.772386


In [86]:
# seperate train,val,test based on given time value ending such as "2017-01-16"
train = spei_multi[spei_multi['time'] < '2017-01-16']
val = spei_multi[(spei_multi['time'] >= '2017-01-16') & (spei_multi['time'] < '2020-01-16')]
test = spei_multi[spei_multi['time'] >= '2020-01-16']

train_metrics = calculate_metrics(train)
val_metrics = calculate_metrics(val)
test_metrics = calculate_metrics(test)
val_metrics

{'rmse': np.float64(0.8542453102643408),
 'mae': np.float64(0.5784158077295029),
 'r2': 0.36106145540920176}

In [90]:
test_metrics

{'rmse': np.float64(1.1547704045318352),
 'mae': np.float64(0.8729438865862684),
 'r2': -0.19084218667194008}

In [87]:
train_single = spei_single[spei_single['time'] < '2017-01-16']
val_single = spei_single[(spei_single['time'] >= '2017-01-16') & (spei_single['time'] < '2020-01-16')]
test_single = spei_single[spei_single['time'] >= '2020-01-16']
train_single_metrics = calculate_metrics(train_single)
val_single_metrics = calculate_metrics(val_single)
test_single_metrics = calculate_metrics(test_single)
val_single_metrics

{'rmse': np.float64(1.0667764054312452),
 'mae': np.float64(0.8586274778240405),
 'r2': 0.003488993640093696}

In [91]:
test_single_metrics

{'rmse': np.float64(1.2535336097504868),
 'mae': np.float64(0.9790235769716297),
 'r2': -0.40337231565601894}

In [88]:
train_single_metrics

{'rmse': np.float64(1.094615789462386),
 'mae': np.float64(0.8663898103727385),
 'r2': -0.21384650377420522}

In [ ]:
# add SPEI ConvLSTM metrics to df_metrics with the same structure
spei_multi_metrics_flat = {
    "train_rmse": train_metrics['rmse'],
    "train_mae": train_metrics['mae'],
    "train_r2": train_metrics['r2'],
    "validation_rmse": val_metrics['rmse'],
    "validation_mae": val_metrics['mae'],
    "validation_r2": val_metrics['r2'],
    "test_rmse": test_metrics['rmse'],
    "test_mae": test_metrics['mae'],
    "test_r2": test_metrics['r2']
}
spei_multi_metrics_flat['folder_name'] = 'SPEI_ConvLSTM-MultiTask'
# Add the SPEI ConvLSTM row to the DataFrame
df_metrics = pd.concat([df_metrics, pd.DataFrame([spei_multi_metrics_flat])])
# Add SPEI Single ConvLSTM metrics to df_metrics with the same structure
spei_single_metrics_flat = {
    "train_rmse": train_single_metrics['rmse'],
    "train_mae": train_single_metrics['mae'],
    "train_r2": train_single_metrics['r2'],
    "validation_rmse": val_single_metrics['rmse'],
    "validation_mae": val_single_metrics['mae'],
    "validation_r2": val_single_metrics['r2'],
    "test_rmse": test_single_metrics['rmse'],
    "test_mae": test_single_metrics['mae'],
    "test_r2": test_single_metrics['r2']
}
spei_single_metrics_flat['folder_name'] = 'SPEI_ConvLSTM-SingleTask'
# Add the SPEI Single ConvLSTM row to the DataFrame
df_metrics = pd.concat([df_metrics, pd.DataFrame([spei_single_metrics_flat])])
# Save the updated DataFrame to CSV
df_metrics.to_csv('results/global_convlstm_improved_full_metrics.csv', index=False)
df_metrics_spei = df_metrics[df_metrics['folder_name'].str.contains('SPEI_')]